In [11]:
!pip install category_encoders

     ---------------------------------------- 81.9/81.9 kB 4.5 MB/s eta 0:00:00


In [12]:
import sqlite3

con=sqlite3. connect('./waitlist.db')
cur=con.cursor()

In [44]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [14]:
df = pd.read_sql_query("SELECT * FROM kidney_waitlist", con)
df.head()

,patient_id,age_cat,dialysis_duration,gender,underlying_disease,blood_group,gestation,prior_transplant,cPRA_cat,HLA_A1,...,HLA_B1,HLA_B2,HLA_DR1,HLA_DR2,DR_00,B_00,A_00,death,transplanted_or_not,waiting_time
0,1,Over60,1.0,M,others,A,No,No,Zero,1,...,44,51,3,7,Heterozygote,Heterozygote,Heterozygote,No,No,13.07
1,2,From18to60,4.0,M,diabetes,A,No,No,Zero,1,...,18,35,11,0,Homozygote,Heterozygote,Heterozygote,Yes,No,68.87
2,3,From18to60,2.0,M,HUS,O,No,No,From50To80,24,...,14,18,1,15,Heterozygote,Heterozygote,Heterozygote,No,No,12.17
3,4,From18to60,17.0,M,diabetes,O,No,Yes,ZeroTo50,24,...,14,18,1,15,Heterozygote,Heterozygote,Heterozygote,No,No,12.17
4,5,Over60,68.0,M,HUS,A,No,No,Zero,24,...,14,27,13,15,Heterozygote,Heterozygote,Heterozygote,Yes,No,6.47


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48153 entries, 0 to 48152
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   patient_id           48153 non-null  int64  
 1   age_cat              48153 non-null  object 
 2   dialysis_duration    48153 non-null  float64
 3   gender               48153 non-null  object 
 4   underlying_disease   48153 non-null  object 
 5   blood_group          48153 non-null  object 
 6   gestation            48153 non-null  object 
 7   prior_transplant     48153 non-null  object 
 8   cPRA_cat             48153 non-null  object 
 9   HLA_A1               48153 non-null  int64  
 10  HLA_A2               48153 non-null  int64  
 11  HLA_B1               48153 non-null  int64  
 12  HLA_B2               48153 non-null  int64  
 13  HLA_DR1              48153 non-null  int64  
 14  HLA_DR2              48153 non-null  int64  
 15  DR_00                48153 non-null 

In [38]:
# separate feature / target
target = df['waiting_time']
feature = df.drop(columns='waiting_time')

# separate train / test
x_train, x_test,y_train, y_test = train_test_split(feature, target,test_size=0.3, random_state=42)

In [39]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33707 entries, 24954 to 15795
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   patient_id           33707 non-null  int64  
 1   age_cat              33707 non-null  object 
 2   dialysis_duration    33707 non-null  float64
 3   gender               33707 non-null  object 
 4   underlying_disease   33707 non-null  object 
 5   blood_group          33707 non-null  object 
 6   gestation            33707 non-null  object 
 7   prior_transplant     33707 non-null  object 
 8   cPRA_cat             33707 non-null  object 
 9   HLA_A1               33707 non-null  int64  
 10  HLA_A2               33707 non-null  int64  
 11  HLA_B1               33707 non-null  int64  
 12  HLA_B2               33707 non-null  int64  
 13  HLA_DR1              33707 non-null  int64  
 14  HLA_DR2              33707 non-null  int64  
 15  DR_00                33707 non-n

In [41]:
y_train.head()

24954    189.07
8388      88.03
19366      3.10
23409     61.97
35466     33.53
Name: waiting_time, dtype: float64

In [46]:
from scipy.stats import randint, uniform

pipe = make_pipeline(
    MinMaxScaler(),
    OneHotEncoder(), 
    TargetEncoder(),
    SimpleImputer(), 
    RandomForestRegressor(random_state=42)
)

dists = {
    'targetencoder__smoothing': [2.,20.,50.,60.,100.,500.,1000.], 
    'targetencoder__min_samples_leaf': randint(1, 10),     
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestregressor__n_estimators': randint(50, 500), 
    'randomforestregressor__max_depth': [5, 10, 15, 20, None], 
    'randomforestregressor__max_features': uniform(0, 1) 
}

clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='neg_mean_absolute_error',  
    verbose=1,
    n_jobs=-1
)

clf.fit(x_train, y_train);

Fitting 3 folds for each of 50 candidates, totalling 150 fits


ValueError: 
All the 150 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\OWNER\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\OWNER\anaconda3\lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\OWNER\anaconda3\lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "C:\Users\OWNER\anaconda3\lib\site-packages\joblib\memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "C:\Users\OWNER\anaconda3\lib\site-packages\sklearn\pipelin